In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import labrotation.file_handling as fh
import h5py
from time import time
import matplotlib.pyplot as plt
import numpy as np
import os
from labrotation import file_handling as fh
from copy import deepcopy
import pandas as pd
import labrotation.two_photon_session as tps
import seaborn as sns
import uuid  # for unique labeling of sessions and coupling arrays (mouse velocity, distance, ...) to sessions in dataframe 
from matplotlib import cm  # colormap
import datadoc_util
from labrotation import two_photon_session as tps

# If exists, load environmental variables from .env file

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

# Set up data documentation directory

In [ ]:
# assumption: inside the documentation folder, the subfolders carry the id of each mouse (not exact necessarily, but they 
# can be identified by the name of the subfolder). 
# Inside the subfolder xy (for mouse xy), xy_grouping.xlsx and xy_segmentation.xlsx can be found.
# xy_grouping.xlsx serves the purpose of finding the recordings belonging together, and has columns:
# folder, nd2, labview, lfp, face_cam_last, nikon_meta, experiment_type, day
# xy_segmentation.xlsx contains frame-by-frame (given by a set of disjoint intervals forming a cover for the whole recording) 
# classification of the events in the recording ("normal", seizure ("sz"), sd wave ("sd_wave") etc.). The columns:
# folder, interval_type, frame_begin, frame_end.

# TODO: write documentation on contents of xlsx files (what the columns are etc.)
if "DATA_DOCU_FOLDER" in env_dict.keys():
    docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    docu_folder = fh.open_dir("Choose folder containing folders for each mouse!")
print(f"Selected folder:\n\t{docu_folder}")

In [ ]:
if "documentation" in os.listdir(docu_folder):
    mouse_folder = os.path.join(docu_folder, "documentation")
else:
    mouse_folder = docu_folder
mouse_names = os.listdir(mouse_folder)
print(f"Mice detected:")
for mouse in mouse_names:
    print(f"\t{mouse}")

In [ ]:
mouse_folder

### Load matlab-2p

In [ ]:
if "MATLAB_2P_FOLDER" in env_dict.keys():
    matlab_2p_folder = env_dict["MATLAB_2P_FOLDER"]
else:
    matlab_2p_folder = fh.open_dir("Choose matlab-2p folder")
print(f"matlab-2p folder set to:\n\t{matlab_2p_folder}")

In [ ]:
df_seg_complete = pd.DataFrame(columns = ["nd2", "interval_type", "frame_begin", "frame_end"])
df_grouping_complete = pd.DataFrame(columns = ["folder", "nd2", "labview", "lfp", "face_cam_last", "nikon_meta", "experiment_type", "mouse_id", "day"])

for mouse_id in mouse_names:
    print(mouse_id)
    seg_fpath = os.path.join(mouse_folder, mouse_id, mouse_id + '_segmentation.xlsx')
    grouping_fpath = os.path.join(mouse_folder, mouse_id, mouse_id + '_grouping.xlsx')
    if os.path.exists(seg_fpath) and os.path.exists(grouping_fpath):
        df_seg = pd.read_excel(seg_fpath)
        df_grouping = pd.read_excel(grouping_fpath)
        df_grouping["mouse_id"] = mouse_id
        # select only tmev, chr2_szsd, chr2_sd, chr2_ctl experiment data first
        df_grouping = df_grouping[df_grouping["experiment_type"].isin(["tmev", "tmev_ctl", "chr2_sd", "chr2_szsd", "chr2_ctl"])]
        # merge into large dataframes
        # print(f"\tseg bef: {len(df_seg_complete['nd2'])}")
        df_seg_complete = pd.concat([df_seg_complete, df_seg])
        # print(f"\tseg aft: {len(df_seg_complete['nd2'])}")
        # print(f"\tgro bef: {len(df_grouping_complete['nd2'])}")
        df_grouping_complete = pd.concat([df_grouping_complete, df_grouping])
        # print(f"\tgro aft: {len(df_grouping_complete['nd2'])}")
    else:
        print(f"Check if you set the correct folder (folder containing all subfolders with mouse names):")
        if not os.path.exists(seg_fpath):
            print(f"\t{seg_fpath} not found")
        if not os.path.exists(grouping_fpath):
            print(f"\t{grouping_fpath} not found")
        

In [ ]:
ddoc = datadoc_util.DataDocumentation(docu_folder)
ddoc.loadDataDoc()

In [ ]:
# take only recordings that were classified as "tmev" (experiment type)
df_seg_complete.where(df_seg_complete["nd2"].isin(df_grouping_complete["nd2"].unique()), inplace=True)
# wrong recording types changed to NaN; drop them
df_seg_complete.dropna(inplace=True)

# Pre-/post-ictal locomotion

In [ ]:
# take only videos with seizure
df_sz_movies = df_seg_complete.groupby("nd2").filter(lambda group: "sz" in group["interval_type"].unique())

## Add uuid

In [ ]:
df_sz_movies = pd.merge(df_sz_movies, ddoc.getNikonFileNameUuid().dropna(), on="nd2")

### Filter for only tmev-type recording

In [ ]:
df_sz_movies["type"] = df_sz_movies.apply(lambda row: df_grouping_complete[df_grouping_complete["uuid"] == row["uuid"]].experiment_type.values[0], axis=1)

In [ ]:
df_sz_movies = df_sz_movies[df_sz_movies["type"] == "tmev"]

### Add previous recording to cases where seizure starts on frame 1 (or not enough baseline before sz start)

In [ ]:
# 3dd896d33a0f42c698228fbe254ebd60 contains seizure from frame 1; previous recording is 21c83d0b69ec4585a9a11f4ce6c24b99
#pd.merge(df_sz_movies, ddoc.getNikonFileNameUuid().dropna(), on="nd2")
# list of (previous recording, sz recording)
uuids_to_add = [("21c83d0b69ec4585a9a11f4ce6c24b99", "3dd896d33a0f42c698228fbe254ebd60"),  # sz starts on frame 1
                ("0bb9054922664ee3a148618e99da1c6a", "44ca941252064dcabb0fe3d24a8dab49")  # sz starts on frame 131
               ]  # pairs of (uuid of previous recording, uuid of sz recording)
for uuid, uuid_sz in uuids_to_add:
    nd2_to_add = df_grouping_complete[df_grouping_complete["uuid"] == uuid].nd2.values[0]
    group_to_add = df_seg_complete[df_seg_complete["nd2"] == nd2_to_add].to_dict()
    group_to_add["uuid"] = uuid
    group_to_add["uuid_sz"] = uuid_sz
    group_to_add = pd.DataFrame(group_to_add)
    group_to_add["type"] = group_to_add.apply(lambda row: df_grouping_complete.loc[df_grouping_complete["uuid"] == row["uuid"]].experiment_type.values[0], axis=1)
    df_sz_movies = pd.concat([df_sz_movies, group_to_add])
    

### Add following recording to cases where not enough aftermath available

In [ ]:
# list of (sz recording, aftermath_recording)
uuids_to_add = [("92062a977958443e83011619b34eabb8", "3cb9934ddcc24cf7a922dca01bdb9448"),  # sz ends on frame 15934/17978
               ]  # pairs of (uuid of previous recording, uuid of sz recording)
for uuid_sz, uuid in uuids_to_add:
    nd2_to_add = df_grouping_complete[df_grouping_complete["uuid"] == uuid].nd2.values[0]
    group_to_add = df_seg_complete[df_seg_complete["nd2"] == nd2_to_add].to_dict()
    group_to_add["uuid"] = uuid
    group_to_add["uuid_sz"] = uuid_sz
    group_to_add = pd.DataFrame(group_to_add)
    group_to_add["type"] = group_to_add.apply(lambda row: df_grouping_complete.loc[df_grouping_complete["uuid"] == row["uuid"]].experiment_type.values[0], axis=1)
    df_sz_movies = pd.concat([df_sz_movies, group_to_add])
    

### Remove recording with not sufficient aftermath recorded
Unfortunately, the video is too short, and no subsequent recordings were made.

In [ ]:
df_sz_movies = df_sz_movies[df_sz_movies["uuid"] != "b8f31023d2c042c2a7f95b54d9807cb7"]

## Add seizure-uuid
See 4 Directionality analysis for original context. This code should be the same as there! (If update needed, need to extract into third file!)
In this analysis, the purpose is to make each seizure unique (to deal with seizures split-up in two videos, for example)

In [ ]:
df_sz_movies["uuid_sz"] = df_sz_movies["uuid"]
# following two recordings contain 1 seizure-sd event. Make first video uuid the uuid_sz
df_sz_movies["uuid_sz"] = df_sz_movies["uuid_sz"].replace( "30dc55d1a5dc4b0286d132e72f208ca6", "65bff16a4cf04930a5cb14f489a8f99b")
# following recordings do not have sz
#qdf = qdf[qdf["uuid_matched"] != "171693d0988c458a96c8198c7b8cfc28"]

In [ ]:
df_sz_intervals = df_sz_movies[df_sz_movies["interval_type"] == "sz"]

In [ ]:
for i_g, g in df_sz_intervals.sort_values(["nd2"]).groupby("uuid_sz"):  # assume seizures cut in two have incremented names
    print(f'{g[g["interval_type"] == "sz"].frame_begin.values}: {g[g["interval_type"] == "sz"].uuid.values}')
    # check if one recording contained several seizures:
    n_seizures = len(g[g["interval_type"] == "sz"].uuid_sz.unique())
    n_recordings = len(g[g["interval_type"] == "sz"].uuid.unique())
    print(f"{n_seizures} sz: {n_recordings} recs")
    
    
    # group by uuid_sz?
    # TODO: need some way to sort by recording starting time too.

In [ ]:
# loop through each seizure. Find recording, find 5 minutes before beginning of sz, during sz + sd, and 5 minutes after end of sz
# 1. for each sz (uuid_sz), find ordered recordings
# 2. find beginning of sz = first recording with a "sz" category
# 3. find uuid of the recording, open loco data, extract locomotion of 5 min before sz begin
# 4. find end of sz = last recording with a "sz" or "sd_wave" category
# 5. find uuid of the recording, open loco data, extract locomotion of 5 min after sz end

In [ ]:
df_sz_intervals["i_sz"] = df_sz_intervals.groupby("uuid").cumcount() + 1

### Sort by recordings (ASSUMPTION: recording indexing is incremental)
Once using groupby uuid_sz, the recordings where the same seizure is present should then be ordered from the first recording it appears in to the last.

In [ ]:
df_sz_intervals = df_sz_intervals.sort_values("nd2")

### Make uuid_sz truly unique
Append the index of the seizure, i.e. uuid -> uuid_1, uuid_2 etc.

In [ ]:
df_sz_intervals["uuid_sz"] = df_sz_intervals.apply(lambda row: row.uuid_sz + "_" +  str(row.i_sz), axis=1)

### Create data structure of intervals
For each seizure, the entry should contain the baseline (5 min), sz+sd, post-sz (5 min) periods. If one period spreads over multiple recordings, then this will be an array.
The data structure:

A dictionary of the uuid_sz values, each has as its value another dictionary, with keys "baseline", "sz", "aftermath". The corresponding values are lists of tuples. Each tuple contains the uuid of the session, the beginning frame (1-indexing) and end frame in that session (both inclusive! i.e. in case of 1 and 10, the segment is 1 to 10 inclusive, the segment having a length of 10 frames). If the list of tuples has more than one entry, it means that the given interval (baseline, sz, aftermath) spreads over multiple sessions (i.e. recordings).

Example:

{

\<uuid_sz1\>: 

{

  "baseline": \[ (\<uuid1\>, \<begin_frame\>, \<end_frame\>), (\<uuid2\>, \<begin_frame\>, \<end_frame\>)  \],

  "sz": \[ ( \<uuid\>, \<begin_frame\>, \<end_frame\> ) \],
  
  "sd": \[ ( \<uuid1\>, \<begin_frame\>, \<end_frame\> ), ... \],
 
  "aftermath": \[ ( \<uuid1\>, \<begin_frame\>, \<end_frame\> ), ... \],
  
  }
  
  
  \<uuid_sz2\>: {...}

}

In [ ]:
RECORDING_FRAMERATE = 15.0  # in Hz
BL_LEN_S = 5*60  # baseline desired (approximate) length, in seconds
AM_LEN_S = 5*60  # aftermath desired (approximate) in seconds

bl_len_frames = RECORDING_FRAMERATE * BL_LEN_S
am_len_frames = RECORDING_FRAMERATE * AM_LEN_S

In [ ]:
sz_intervals_dict = dict()
for uuid_sz, g in df_sz_intervals.sort_values("nd2").groupby(["uuid_sz"]):
    sz_entry = []
    sd_entry = []  # keep sd separate from sz at this point. The plan is to combine them when looking at locomotion, though.
    bl_entry = []  # baseline
    am_entry = []  # aftermath
    
    # 1. get sz data.
    assert len(g) <= 2  # for now, only deal with cases where segment is in single recording or in two recordings. 
    # If more recordings (highly unlikely), need to switch to array-based approach
    g_sz = g[(g["interval_type"] == "sz")]
    sz_begin_uuid = g_sz[g_sz["uuid_sz"] == uuid_sz].iloc[0].uuid
    sz_end_uuid = g_sz[g_sz["uuid_sz"] == uuid_sz].iloc[-1].uuid  # assume len(g["sz"]) == 1 or == 2!!!
    # Assume only seizure is split up, the baseline and aftermath never span over two recordings
    sz_begin_session = df_grouping_complete.loc[df_grouping_complete["uuid"] == sz_begin_uuid]
    sz_end_session = df_grouping_complete.loc[df_grouping_complete["uuid"] == sz_end_uuid]
    # define seizure begin and end frames (might be from different recordings!) to aid acquiring baseline and aftermath frames
    sz_begin_frame = g_sz.iloc[0].frame_begin
    sz_end_frame = g_sz.iloc[-1].frame_end
    
    # TODO: add sd_wave to these intervals as well
    if sz_begin_uuid == sz_end_uuid:  # seizure entirely in a single recording
        sz_entry.append((sz_begin_uuid, sz_begin_frame, sz_end_frame))
    else:
        assert len(g) == 2  # make sure only up to 2 recordings, as this is the only other case handled
        sz_entry.append((sz_begin_uuid, sz_begin_frame, g.iloc[0].frame_end))
        sz_entry.append((sz_end_uuid, g.iloc[1].frame_begin, sz_end_frame))
    
    # assumption: sd wave comes after seizure, and both waves are in same movie as sz end.
    last_frame = df_sz_movies[(df_sz_movies["uuid"] == sz_end_uuid)].frame_end.max() 
    current_frame = sz_end_frame + 1
    if not sz_end_frame == last_frame:  # more segments after seizure
        for i_row, row in df_sz_movies[(df_sz_movies["uuid"] == sz_end_uuid) & ((df_sz_movies["interval_type"] == "sd_wave") | (df_sz_movies["interval_type"] == "sd_wave_cx"))].iterrows():
            if row.interval_type == "normal":
                break
            elif row.interval_type == "sd_wave":
                if row.frame_begin == current_frame:  # look for segment that comes just after sz (or sd wave)
                    sd_entry.append((sz_end_uuid, row.frame_begin, row.frame_end))
                    current_frame = row.frame_end + 1
                
    else:
        raise Exception(f"Rare condition: seizure seems to have ended just at the end of the recording. Might be a bug (next recording containing rest incl. sd waves not included in list)?")
        
    
    
    
    # get baseline
    if sz_begin_frame == 1:  # seizure begins with recording; happens at least once
        print(f"Frame 1 sz: {uuid_sz}")
        # need to take previous recording end as baseline.
        # Just assume that the very first recording is the previous recording. It should be, in any case.
        bl_uuid = g.iloc[0].uuid
        print(f"\tBL uuid: {bl_uuid}")
        len_bl_movie = df_sz_movies[(df_sz_movies["uuid"] == bl_uuid) & (df_sz_movies["interval_type"] == "normal")].frame_end.max()
        bl_end_frame = len_bl_movie
        bl_begin_frame = bl_end_frame - bl_len_frames
    else:  # if seizure was not in beginning, assume we have enough frames before sz start in same video (in this version of the code)
        bl_uuid = sz_begin_uuid
        bl_end_frame = sz_begin_frame - 1
        bl_begin_frame = sz_begin_frame - bl_len_frames
    if bl_begin_frame < 1:  # not enough baseline available in recording
        # TODO: modify exception to warning? No way to add baseline in front anyway... Recordings are never
        # directly after another; it might be possible to take previous recording somehow, and 
        print(f"Baseline: Not enough baseline available for {sz_begin_uuid}. Seizure begins on frame {sz_begin_frame}. Need at least {bl_len_frames} frames before for {BL_LEN_S} seconds of baseline. Using part of previous recording")
        bl_end_frame_sz_movie = sz_begin_frame - 1  # 1 to (sz_begin_frame - 1) is second part of baseline
        bl_begin_frame_sz_movie = 1
        bl_len_rest = bl_len_frames - bl_end_frame_sz_movie  # this many frames to take from previous recording
        
        # assume movie before sz is present, and the first recording in the ordered-by-filename list 
        bl_uuid = g.iloc[0].uuid
        len_bl_movie = df_sz_movies[(df_sz_movies["uuid"] == bl_uuid) & (df_sz_movies["interval_type"] == "normal")].frame_end.max()
        bl_end_frame_prev_movie = len_bl_movie
        bl_begin_frame_prev_movie = len_bl_movie - bl_len_rest
        
        bl_entry.append((bl_uuid, int(bl_begin_frame_prev_movie), int(bl_end_frame_prev_movie)))
        bl_entry.append((sz_begin_uuid, int(bl_begin_frame_sz_movie), int(bl_end_frame_sz_movie)))
    else:
        bl_entry.append((bl_uuid, int(bl_begin_frame), int(bl_end_frame)))
    # get aftermath
    am_uuid = sz_end_uuid
    am_begin_frame = sz_end_frame + 1
    am_end_frame = sz_end_frame + am_len_frames
    # test if aftermath would fall outside last "normal" interval in recording:
    # TODO: aftermath should be usually either normal or sd_extinction!
    len_sz_movie = df_sz_movies[(df_sz_movies["uuid"] == am_uuid) & (df_sz_movies["interval_type"] == "normal")].frame_end.max()
    if am_end_frame > len_sz_movie:
        print(f"Aftermath: Not enough aftermath available for {sz_end_uuid}. Seizure ends on frame {sz_end_frame}, total {len_sz_movie} frames. Need {am_len_frames} frames after for {AM_LEN_S} seconds of aftermath")
        am_len_sz_movie = len_sz_movie - am_begin_frame + 1  # length of aftermath in sz movie
        am_len_next_movie = am_len_frames - am_len_sz_movie  # length of aftermath we need from next movie
        # take last movie in group, , take first am_len_next_movie frames, append both intervals
        next_movie_uuid = g.iloc[-1].uuid  # assume last movie in group is aftermath movie
        am_begin_frame_next_movie = 1
        am_end_frame_next_movie = am_len_next_movie
        am_entry.append((am_uuid, int(am_begin_frame), int(len_sz_movie)))
        am_entry.append((next_movie_uuid, int(am_begin_frame_next_movie), int(am_end_frame_next_movie)))
    else:
        am_entry.append((am_uuid, int(am_begin_frame), int(am_end_frame)))
    sz_intervals_dict[uuid_sz] = {"sz": sz_entry, "sd": sd_entry, "baseline": bl_entry, "aftermath": am_entry}

# Create dictionary of locomotion data
{uuid1: \{ "baseline": \[ loco data \], "sz": \[ loco data \], "aftermath": \[ loco data \] \}, uuid2: {...}, ...}

In [ ]:
fluolv_dir = fh.open_dir("Choose FluLV directory (directory with fluorescence & labview extracted)")

In [ ]:
loco_dict = dict()
types_list = ["baseline", "sz", "sd", "aftermath"]
for uuid_sz, intervals in sz_intervals_dict.items():
    # open locomotion data for each uuid
    # for each type (bl, sz, am), for each fragment of the interval, extract the locomotion data
    # dictionary looks like:
    # "<uuid1>": { 
    #   "baseline": {
    #     "speed": [[speed_segment1], [speed_segment2], ...],
    #     "running_binary": [[running_binary_segment1], ...],
    #     "total_distance_mm": [[total_distance_mm_segment1], ...],
    #     "time_ms": [[time_ms_segment1], ...],
    #     },
    #   "sz":
    #      {...},
    #   "aftermath":
    #      {...},
    #   },
    # "<uuid2>": {...},
    # ...
    
    # add uuid entry
    loco_dict[uuid_sz] = dict()
    for interval_type in types_list:  # baseline, sz, aftermath
        #add baseline, sz, aftermath entry
        loco_dict[uuid_sz][interval_type] = dict()
        # fill baseline entry with empty arrays
        loco_dict[uuid_sz][interval_type]["speed"] = []
        loco_dict[uuid_sz][interval_type]["running_binary"] = []
        loco_dict[uuid_sz][interval_type]["total_distance_mm"] = []
        loco_dict[uuid_sz][interval_type]["time_ms"] = []
        tuples = intervals[interval_type]
        for uuid, begin_frame, end_frame in tuples:
            fpath = os.path.join(fluolv_dir, uuid + "_FluLoco.h5")
            if not os.path.exists(fpath):
                raise Error(f"Error: {fpath} does not exist!")
            with h5py.File(fpath, "r") as f:
                speed = f["speed_scn"][begin_frame-1:end_frame]  # _scn data are matched to Nikon frames; 0-indexing!
                running_binary = f["running_scn"][begin_frame-1:end_frame]#
                total_distance_mm = f["totdistscn"][begin_frame-1:end_frame]#[begin_frame-1:end_frame]
                time_ms = f["tsscn"][begin_frame-1:end_frame]#[begin_frame-1:end_frame]
                loco_dict[uuid_sz][interval_type]["speed"].append(speed)
                loco_dict[uuid_sz][interval_type]["running_binary"].append(running_binary)
                loco_dict[uuid_sz][interval_type]["total_distance_mm"].append(total_distance_mm)
                loco_dict[uuid_sz][interval_type]["time_ms"].append(time_ms)  

# Create summary statistic for movement, put into dataframe
Column names: uuid_sz, baseline_totdist, sz_totdist, aftermath_totdist

### Create uuid_sz - mouse ID relation
Useful for grouping per mouse

In [ ]:
df_id_uuid = ddoc.getIdUuid()

In [ ]:
dict_uuid_sz_id = dict() 
for uuid_sz in loco_dict.keys():
    uuid = uuid_sz.split("_")[0]
    mouse_id = df_id_uuid[df_id_uuid["uuid"] == uuid].mouse_id.values[0]
    dict_uuid_sz_id[uuid_sz] = mouse_id 

### Calculate total distance run

In [ ]:
total_dist_dict = {segment_type: dict() for segment_type in types_list}
for uuid in loco_dict.keys():
    for interval_type in types_list:
        tot_dist_mm = 0.0
        for segment in loco_dict[uuid][interval_type]["total_distance_mm"]:  # interval might be broken up in multiple recordings/segments
            tot_dist_mm += abs(segment[-1] - segment[0])
        total_dist_dict[interval_type][uuid] = tot_dist_mm

In [ ]:
df_total_dist_mm = pd.DataFrame.from_dict(total_dist_dict).reset_index().rename(columns={"index": "uuid_sz"})

### Calculate percent of time spent running

In [ ]:
running_binary_dict = {segment_type: dict() for segment_type in types_list}
for uuid in loco_dict.keys():
    for interval_type in types_list:
        running_frames = 0
        all_frames = 0
        for segment in loco_dict[uuid][interval_type]["running_binary"]:  # interval might be broken up in multiple recordings/segments
            running_frames += sum(segment)
            all_frames += len(segment)
        if all_frames > 0:
            running_binary_dict[interval_type][uuid] = float(running_frames)/all_frames
        else:
            assert running_frames == 0.0
            running_binary_dict[interval_type][uuid] = np.nan

### Combine sz + sd into single "seizure" interval

In [ ]:
df_total_dist_mm["seizure"] = df_total_dist_mm["sz"] + df_total_dist_mm["sd"]

In [ ]:
df_total_dist_mm["mouse_id"] = df_total_dist_mm.apply(lambda row: dict_uuid_sz_id[row["uuid_sz"]], axis=1)

## Reshape dataframe into (uuid_sz, total_distance, interval_type) format

In [ ]:
df_total_dist_melted = pd.melt(df_total_dist_mm, id_vars=["uuid_sz", "mouse_id"], value_vars=["baseline", "seizure", "aftermath"], var_name="type", value_name="total_dist_mm")

## Exploratory plotting
Two dataframes are helpful: df_total_dist_mm, df_total_dist_melted. First has columns "uuid_sz", "baseline", "seizure", "aftermath" (+ "sz", "sd", which are combined into "seizure"). Second has columns "uuid_sz", "interval_type", "total_dist_mm"

In [ ]:
fig = plt.figure(figsize=(12,12))
sns.pairplot(data=df_total_dist_melted, y_vars="total_dist_mm", x_vars = "type", height=5, aspect=.8)
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,12))
sns.lineplot(data=df_total_dist_melted, x="type", y="total_dist_mm", hue="mouse_id", size="uuid_sz", sizes=(2,2), errorbar=None, legend=False, markers=True)
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,12))
sns.lineplot(data=df_total_dist_melted[df_total_dist_melted["type"] != "seizure"], x="type", y="total_dist_mm", hue="mouse_id", size="uuid_sz", sizes=(2,2), errorbar=None, legend=False)
#plt.savefig('D:\\Downloads\\foo.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,12))
sns.pointplot(data=df_total_dist_melted[df_total_dist_melted["type"] != "seizure"], x="type", y="total_dist_mm", hue="mouse_id", errorbar=None)
plt.show()